<a href="https://www.bigdatauniversity.com"><img src="https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width="400" align="center"></a>

<h1 align=center><font size="5"> Coursework for week 3 of IBM Data Science course </font></h1>

In this notebook homework assignment for week 3 is completed, which is: __"Build the code to scrape the Wikipedia page "List of postal codes of Canada: M"__ , and after that explore and cluster the neighborhoods in Toronto.

The tutor's instructions are: *Use the Notebook to build the code to scrape the [Wikipedia page,](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M), in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown below: (...)*

### __This notebook is prepared by Vladimir Belobragin.__
Please, follow the last version of the notebook in Github repository at: https://github.com/vbelobragin/Coursera_Capstone.git 

<h3 align=left><font size="3"> 1.Scrape the Wikipedia page "List of postal codes of Canada: M" </font></h3>


At first, import libraries.

In [187]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

from bs4 import BeautifulSoup # import recommended html parsing library

print('Libraries imported.')

Libraries imported.


Install visualization library, Folium, to visualize the results:

In [188]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.18.1                     py_0    conda-forge
Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


Create the data drame object:

In [189]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
article = soup.find('table', class_='wikitable sortable')

table_list = []
for rows in article.find_all('td'):
    row = rows.text
    row = row.replace('\n', '')
    table_list.append(row) 
           


Define the dataframe object:

In [190]:
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 
df=[]
df=pd.DataFrame(columns=column_names)

Paste the codes to the dataframe and drop non-assighned boroughs:

In [191]:
df.iloc[:,0]=table_list[::3]
df.iloc[:,1]=table_list[1::3]
df.iloc[:,2]=table_list[2::3]
df.replace("Not assigned", np.nan, inplace = True)
df.dropna(subset=["Borough"], axis=0, inplace = True)
df.reset_index(drop=True, inplace=True)

Now assign to the non-assighned neighborhoods names of boroughs:

In [192]:
for i in range(0, df.shape[0]):
    if pd.isnull(df.loc[i,'Neighborhood']):
        df.replace(df.loc[i,'Neighborhood'], df.loc[i,'Borough'],inplace=True)
df.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


Combine rows with the same code into one row and separate them with a comma:

In [193]:
df['Neighborhood'] = df[['PostalCode','Borough','Neighborhood']].groupby(['PostalCode','Borough'])['Neighborhood'].transform(lambda x: ','.join(x)) 
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)

df.head(15)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson,Garden District"


Now get the final shape of the DataFrame:

In [194]:
df.shape

(103, 3)

<h3 align=left><font size="3"> 2. Load a csv file that has the geographical coordinates of each postal code </font></h3>

Given that package Geocoder is DEFINITELY very unreliable, use a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [195]:
TorontoCoord=df
TorontoCoord1=pd.read_csv("http://cocl.us/Geospatial_data")
TorontoCoord[['Latitude', 'Longitude']]=TorontoCoord1[['Latitude', 'Longitude']]
TorontoCoord.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.763573,-79.188711
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.770992,-79.216917
4,M7A,Queen's Park,Queen's Park,43.773136,-79.239476


In [196]:
TorontoCoord.shape

(103, 5)

<h3 align=left><font size="3"> 3. Map of neighborhoods in Toronto.  </font></h3> 

Get the geographical coordinates of Toronto:

In [197]:
address = 'Toronto Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


Create a map of Toronto with neighborhoods superimposed on top:

In [198]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(TorontoCoord['Latitude'], TorontoCoord['Longitude'], TorontoCoord['Borough'], TorontoCoord['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h3 align=left><font size="3"> 4. Get the list of venues in Toronto.  </font></h3> 

Unfortunately (maybe, FORTUNATELY) Toronto is not as big city as New York. There are only 103 Boroughs in Toronto. Moreover, we do not have coordinates of Neighborhoods in Toronto, only lattitude and longitude of the Postal Codes zones. So, we can only cluster neighborhoods based on the postal codes.

<h3 align=left><font size="2"> Define Foursquare Credentials and Version </font></h3>

__Please be aware, that for privacy purpose variable CLIENT_SECRET is omitted from the GitHub version of the notebook.__ 

In [199]:
CLIENT_ID = 'HSB35PAX0VVRGDIW042W2DIQZ53JRLS2GF3O4X4Y3QWPP5RI' # your Foursquare ID
CLIENT_SECRET = '**********' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET: **********')

Your credentails:
CLIENT_ID: HSB35PAX0VVRGDIW042W2DIQZ53JRLS2GF3O4X4Y3QWPP5RI
CLIENT_SECRET: **********


Borrow the get_category_type function from the Foursquare lab.

In [200]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

Create a function to repeat the process to get the top 50 venues that are in every neighborhood within a radius of 500 meters - for all the neighborhoods in Toronto (based on post codes):

In [201]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
                    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now write the code to run the above function on each neighborhood and create a new dataframe called Toronto_venues.

In [202]:
Toronto_venues = getNearbyVenues(names=TorontoCoord['Neighborhood'],
                                   latitudes=TorontoCoord['Latitude'],
                                   longitudes=TorontoCoord['Longitude']
                                  )

Parkwoods
Victoria Village
Harbourfront,Regent Park
Lawrence Heights,Lawrence Manor
Queen's Park
Islington Avenue
Rouge,Malvern
Don Mills North
Woodbine Gardens,Parkview Hill
Ryerson,Garden District
Glencairn
Cloverdale,Islington,Martin Grove,Princess Gardens,West Deane Park
Highland Creek,Rouge Hill,Port Union
Flemingdon Park,Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens,Eringate,Markland Wood,Old Burnhamthorpe
Guildwood,Morningside,West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor,Downsview North,Wilson Heights
Thorncliffe Park
Adelaide,King,Richmond
Dovercourt Village,Dufferin
Scarborough Village
Fairview,Henry Farm,Oriole
Northwood Park,York University
East Toronto
Harbourfront East,Toronto Islands,Union Station
Little Portugal,Trinity
East Birchmount Park,Ionview,Kennedy Park
Bayview Village
CFB Toronto,Downsview East
The Danforth West,Riverdale
Design E

In [203]:
print(Toronto_venues.shape)
print('Note: there are {} uniques categories of Neighborhood.'.format(len(Toronto_venues['Neighborhood'].unique())))
Toronto_venues.head()


(2243, 7)
Note: there are 101 uniques categories of Neighborhood.


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,Victoria Village,43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,Victoria Village,43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
3,"Harbourfront,Regent Park",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
4,"Harbourfront,Regent Park",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store


As we see, we got 1697 venues, which is a reasonable number of venues for clustering purpose.

Let's check how many venues were returned for each neighborhood

In [204]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",3,3,3,3,3,3
Agincourt,19,19,19,19,19,19
"Agincourt North,L'Amoreaux East,Milliken,Steeles East",42,42,42,42,42,42
"Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown",10,10,10,10,10,10
"Bathurst Manor,Downsview North,Wilson Heights",16,16,16,16,16,16
Bayview Village,17,17,17,17,17,17
"Bedford Park,Lawrence Manor East",100,100,100,100,100,100
Berczy Park,2,2,2,2,2,2
"Birch Cliff,Cliffside West",100,100,100,100,100,100


__Let's find out how many unique categories can be curated from all the returned venues__

In [205]:
Toronto_venues[Toronto_venues['Venue Category']=='Neighborhood']

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
303,"Little Portugal,Trinity",43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
427,"Brockton,Exhibition Place,Parkdale Village",43.659526,-79.340923,Leslieville,43.662070,-79.337856,Neighborhood
1079,"Birch Cliff,Cliffside West",43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood
1166,Willowdale South,43.640816,-79.381752,Harbourfront,43.639526,-79.380688,Neighborhood


In [206]:
Toronto_venues.loc[{292,416,893,930},'Venue Category']='Neighborhood1'
#Toronto_venues[Toronto_venues['Venue Category']=='Neighborhood1']

<h3 align=left><font size="3"> 4. Analyze Neighborhoods in Toronto.  </font></h3> 

In [207]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move _Neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head(10)

,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood,Neighborhood1,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wing

And let's examine the new dataframe size.

In [208]:
Toronto_onehot.shape

(2243, 280)

__Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category__

In [209]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motel,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Neighborhood1,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Paper / Office Supplies Store,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wing

__Let's confirm the new size__

In [210]:
Toronto_grouped.shape

(101, 280)

__Let's print each neighborhood along with the top 5 most common venues__

In [211]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    #print(temp)
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')
   

----Adelaide,King,Richmond----
                venue  freq
0            Bus Stop  0.33
1             Airport  0.33
2                Park  0.33
3         Yoga Studio  0.00
4  Miscellaneous Shop  0.00


----Agincourt----
            venue  freq
0  Breakfast Spot  0.11
1     Coffee Shop  0.11
2            Café  0.11
3         Stadium  0.05
4          Bakery  0.05


----Agincourt North,L'Amoreaux East,Milliken,Steeles East----
                 venue  freq
0          Coffee Shop  0.24
1                  Gym  0.05
2                Diner  0.05
3  Japanese Restaurant  0.05
4     Sushi Restaurant  0.05


----Albion Gardens,Beaumond Heights,Humbergate,Jamestown,Mount Olive,Silverstone,South Steeles,Thistletown----
          venue  freq
0   Pizza Place   0.2
1      Pharmacy   0.1
2  Skating Rink   0.1
3           Gym   0.1
4   Coffee Shop   0.1


----Bathurst Manor,Downsview North,Wilson Heights----
                venue  freq
0         Coffee Shop  0.12
1       Deli / Bodega  0.06
2             

__Let's put that into a pandas dataframe__

First, let's write a function to sort the venues in descending order.

In [212]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [213]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Park,Airport,Bus Stop,Event Space,Ethiopian Restaurant,Falafel Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
1,Agincourt,Coffee Shop,Breakfast Spot,Café,Climbing Gym,Bar,Italian Restaurant,Stadium,Caribbean Restaurant,Furniture / Home Store,Burrito Place
2,"Agincourt North,L'Amoreaux East,Milliken,Steel...",Coffee Shop,Sushi Restaurant,Diner,Gym,Japanese Restaurant,Café,Bar,Portuguese Restaurant,Spa,Italian Restaurant
3,"Albion Gardens,Beaumond Heights,Humbergate,Jam...",Pizza Place,Gym,Athletics & Sports,Pharmacy,Pool,Pub,Sandwich Place,Skating Rink,Coffee Shop,General Travel
4,"Bathurst Manor,Downsview North,Wilson Heights",Coffee Shop,Frozen Yogurt Shop,Fast Food Restaurant,Sandwich Place,Bridal Shop,Diner,Bank,Restaurant,Deli / Bodega,Sushi Restaurant


<h3 align=left><font size="3"> 5. Cluster Neighborhoods.  </font></h3> 

Run k-means to cluster the neighborhood into 6 clusters.

In [214]:
# set number of clusters
kclusters = 4

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:100]


array([2, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 2, 2, 3, 3, 3, 3, 3, 3, 3, 2, 3, 2,
       3, 3, 3, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 1, 3, 3, 2, 3, 3, 2, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 2, 3, 0, 3, 3, 3, 3, 0, 3, 3, 3, 3, 2, 3, 3, 3, 2, 2, 3, 3,
       3, 3, 2, 3, 2, 3, 3, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [215]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = TorontoCoord

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Toronto_merged.dropna(subset=["Cluster Labels"], axis=0, inplace = True)
Toronto_merged['Cluster Labels'] = Toronto_merged['Cluster Labels'].astype(int)

Toronto_merged.describe()
#Toronto_merged.head() # check the last columns!

,Latitude,Longitude,Cluster Labels
count,101.000000,101.000000,101.000000
mean,43.703669,-79.397712,2.693069
std,0.051186,0.095271,0.674427
min,43.602414,-79.615819,0.000000
25%,43.659526,-79.464763,3.000000
50%,43.696948,-79.388790,3.000000
75%,43.744734,-79.340923,3.000000
max,43.815252,-79.160497,3.000000


Finally, let's visualize the resulting clusters

In [216]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h3 align=left><font size="3"> 6. Examine Clusters.  </font></h3> 

Examine each cluster and determine the discriminating venue categories that distinguish each cluster. 

__Cluster 1__

In [217]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Scarborough,0,Korean Restaurant,Baseball Field,Food Truck,Home Service,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
91,Downtown Toronto,0,Baseball Field,Women's Store,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Eastern European Restaurant
97,Downtown Toronto,0,Paper / Office Supplies Store,Baseball Field,Women's Store,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore


This is definetly, a baseball funs cluster...

__Cluster 2__

In [218]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Etobicoke,1,Convenience Store,Playground,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Women's Store
14,East York,1,Park,Playground,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
48,Downtown Toronto,1,Playground,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


This is a playground cluster. Nice for children!

__Cluster 3__

In [219]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,2,Fast Food Restaurant,Women's Store,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant
20,Downtown Toronto,2,Park,Cafeteria,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
23,East York,2,Park,Bank,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Dessert Shop
25,Downtown Toronto,2,Fast Food Restaurant,Food & Drink Shop,Bus Stop,Park,Gym,Department Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
30,Downtown Toronto,2,Park,Airport,Bus Stop,Event Space,Ethiopian Restaurant,Falafel Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
40,North York,2,Park,Coffee Shop,Convenience Store,Falafel Restaurant,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dessert Shop
44,Scarborough,2,Park,Bus Line,Swim School,Dim Sum Restaurant,Farmers Market,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
50,North York,2,Park,Trail,Playground,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
63,York,2,Garden,Drugstore,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Department Store
64,York,2,Jewelry Store,Trail,Park,Sushi Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop


__Cluster 4__

In [220]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,3,Bar,History Museum,Women's Store,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
2,Downtown Toronto,3,Breakfast Spot,Electronics Store,Spa,Pizza Place,Mexican Restaurant,Rental Car Location,Medical Center,Doner Restaurant,Dim Sum Restaurant,Diner
3,North York,3,Coffee Shop,Korean Restaurant,Women's Store,Dumpling Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore
4,Queen's Park,3,Lounge,Caribbean Restaurant,Fried Chicken Joint,Bakery,Bank,Thai Restaurant,Athletics & Sports,Hakka Restaurant,Electronics Store,Eastern European Restaurant
6,Scarborough,3,Discount Store,Convenience Store,Train Station,Department Store,Coffee Shop,Event Space,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant
7,North York,3,Bus Line,Bakery,Fast Food Restaurant,Intersection,Business Service,Bus Station,Metro Station,Soccer Field,Park,German Restaurant
8,East York,3,Movie Theater,Motel,American Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop
9,Downtown Toronto,3,College Stadium,Skating Rink,Café,General Entertainment,Women's Store,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant
10,North York,3,Indian Restaurant,Vietnamese Restaurant,Latin American Restaurant,Chinese Restaurant,Pet Store,Empanada Restaurant,Electronics Store,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant
11,Etobicoke,3,Middle Eastern Restaurant,Sandwich Place,Breakfast Spot,Bakery,Auto Garage,Shopping Mall,Electronics Store,Empanada Restaurant,Eastern European Restaurant,Dumpling Restaurant


### THANK YOU.

In [ ]:
DisableHTML5Autoplay_Initialize